# Self Drving: Vehicle Detection

### Enviroment Setup

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import random
import itertools
import os
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

from skimage.feature import hog

from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


random.seed(42)
VEHICLE_DIR = "./train/vehicles/"
NON_VEHICLE_DIR = "./train/non-vehicles/"
TEST_IMAGE_DIR = "./test_images/"

### Helper Functions 

In [ ]:
def read_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = read_image(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows
    

def create_threshold_heatmap(image, bbox_list, threshold):
    
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)    
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    
    heatmap[heatmap <= threshold] = 0
    # Return updated heatmap
    return heatmap# Iterate through list of bboxes


### Resource Loading 

In [ ]:
vehicle_image_path_list = glob.glob(VEHICLE_DIR + "./*/*.png")
non_vehicle_image_path_list = glob.glob(NON_VEHICLE_DIR + "./*/*.png")

print("vehicle image count: " + str(len(vehicle_image_path_list)) + " non vehicle image count: " + str(len(non_vehicle_image_path_list)))

#example output
for i in range(3):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
    f.tight_layout()
    vehicle_image = read_image(random.choice(vehicle_image_path_list))
    non_vehicle_image = read_image(random.choice(non_vehicle_image_path_list))
    ax1.imshow(vehicle_image)
    ax1.set_title("vehicle " + str(vehicle_image.shape))
    ax2.imshow(non_vehicle_image)
    ax2.set_title("non vehicle " + str(non_vehicle_image.shape))

plt.show()

### Histogram of Oriented Gradients (HOG)

#### Extracted HOG Features from the Training Images

In [ ]:
COLOR_CHANNEL_DICT = {"HLS:S": (cv2.COLOR_RGB2HLS, 2), "YCrCb:Y": (cv2.COLOR_RGB2YCrCb, 0), "YCrCb:Cr": (cv2.COLOR_RGB2YCrCb, 1)}

orientations = [8, 9]
pixels_per_cell = [6, 8]
cells_per_block = [2, 3]

parameter_combination_list = list(itertools.product(orientations, pixels_per_cell, cells_per_block))

car_image_path = random.choice(vehicle_image_path_list)
not_car_image_path = random.choice(non_vehicle_image_path_list)

for color_channel in COLOR_CHANNEL_DICT.keys():
    print(color_channel)
    car_image = read_image(car_image_path)
    not_car_image = read_image(not_car_image_path)
    
    color_space = COLOR_CHANNEL_DICT[color_channel][0]
    color_channel_index = COLOR_CHANNEL_DICT[color_channel][1] 
    
    car_image = cv2.cvtColor(car_image,color_space)[:,:,color_channel_index]
    not_car_image = cv2.cvtColor(not_car_image, color_space)[:,:,color_channel_index]
    
    for parameter_combination in parameter_combination_list: 
        car_features, car_hog_image = get_hog_features(car_image, parameter_combination[0], parameter_combination[1], parameter_combination[2], 
                        vis=True, feature_vec=True)
        
        not_car_features, not_car_hog_image = get_hog_features(not_car_image, parameter_combination[0], parameter_combination[1], parameter_combination[2], 
                        vis=True, feature_vec=True)
        
        f, axes = plt.subplots(1, 4, figsize=(16, 8))
        
        axes = axes.ravel()
        axes[0].imshow(car_image,cmap='gray')
        axes[0].set_title("car in " + color_channel)
        
        axes[1].imshow(car_hog_image,cmap='gray')
        axes[1].set_title("car HOG \n (orientations,pixels_per_cell, cells_per_block) = \n" + str(parameter_combination))
        
        axes[2].imshow(not_car_image, cmap='gray')
        axes[2].set_title("not car in " + color_channel)
        
        axes[3].imshow(not_car_hog_image,cmap='gray')
        axes[3].set_title("not car HOG\n (orientations,pixels_per_cell, cells_per_block) = \n" + str(parameter_combination))
        
        plt.show()
        

### SVM Classfier Trainning

In [ ]:

color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = False # Spatial features on or off
hist_feat = False # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()


vehicle_features = extract_features(vehicle_image_path_list, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

non_vehicle_features = extract_features(non_vehicle_image_path_list, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

print("vehicle image count: " + str(len(vehicle_features)) + " , non vehicle image count: " + str(len(non_vehicle_features)))

X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64) 
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('training feature vector length:', len(X_train[0]))

parameter_dict = {'kernel':('linear','rbf'), 'C':[0.1,1,10]}
svr = SVC()
clf = GridSearchCV(svr, parameter_dict, verbose=10)
clf.fit(X_train, y_train)

print("Best Fit Parameter: ", clf.best_estimator_)
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))

    
    

### Sliding Window Search

In [ ]:
test_image_path_list  = glob.glob(TEST_IMAGE_DIR + '/*.jpg')
#print(test_image_path_list)

window_size = [32, 64, 128]
overlap = [0.25, 0.5, 0.75]
parameter_combination_list = list(itertools.product(window_size, overlap))


for test_image_path in test_image_path_list:
    test_image = read_image(test_image_path)
    draw_image = np.copy(test_image)
    
    f, axes = plt.subplots(3, 3, figsize=(16, 16))
    f.tight_layout()
    axes = axes.ravel()
    index = 0
    for parameter in parameter_combination_list:
        #print(parameter)
        window_size = parameter[0]
        overlap_ratio = parameter[1]
        windows = slide_window(test_image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                        xy_window=(window_size, window_size), xy_overlap=(overlap_ratio, overlap_ratio))
        
        hot_windows = search_windows(test_image, windows, clf, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
    
        window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
        axes[index].imshow(window_img)
        axes[index].set_title(test_image_path.split(os.sep)[-1] + " window size = " + str(window_size) 
                             + " overlap = " + str(overlap_ratio))
        index += 1

    plt.show()